In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.
/Users/yaqiongzhang/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'top_model_weights_path.h5'

In [3]:
epochs = 50
batch_size = 16
split=0.9

In [4]:
X_train=np.load('X_train.npy')
target_train=np.load('target_train.npy')
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train[:1604], train_size=split)

In [5]:
base_model = applications.VGG16(weights='imagenet', include_top=False,input_shape=(75,75,3))
print('Model loaded.')

Model loaded.


In [6]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [7]:
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

In [8]:
# add the model on top of the convolutional base
#model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

In [9]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

In [13]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-6, momentum=0.5),
              metrics=['accuracy'])

In [14]:
def get_callbacks(filepath, patience):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

file_path = "model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

In [15]:
history = model.fit(
    X_train_cv, y_train_cv,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks)

Train on 1443 samples, validate on 161 samples
Epoch 1/50
1443/1443 [==============================] - 128s 89ms/step - loss: 0.2490 - acc: 0.9238 - val_loss: 0.1914 - val_acc: 0.9565
Epoch 2/50
1443/1443 [==============================] - 129s 89ms/step - loss: 0.2308 - acc: 0.9328 - val_loss: 0.1914 - val_acc: 0.9565
Epoch 3/50
1443/1443 [==============================] - 133s 92ms/step - loss: 0.2366 - acc: 0.9349 - val_loss: 0.1914 - val_acc: 0.9565
Epoch 4/50
1443/1443 [==============================] - 134s 93ms/step - loss: 0.2561 - acc: 0.9328 - val_loss: 0.1914 - val_acc: 0.9565
Epoch 5/50
1443/1443 [==============================] - 135s 94ms/step - loss: 0.2419 - acc: 0.9286 - val_loss: 0.1914 - val_acc: 0.9565
Epoch 6/50
1443/1443 [==============================] - 128s 88ms/step - loss: 0.2648 - acc: 0.9286 - val_loss: 0.1914 - val_acc: 0.9565


In [ ]:
model.load_weights(filepath=file_path)
score = model.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
X_test=np.load('X_test.npy')
predicted_test=model.predict(X_test,verbose=1)

In [ ]:
import pandas as pd
submission = pd.DataFrame()
test_id=np.load('test_id.npy')
submission['id']=test_id
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

In [ ]:
submission
